In [38]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime

### Chequeado Fact Checking

In [ ]:
class FactCheckScraper():
  def get_url(self,page):
    return f'https://chequeado.com/notas/?current-page={page}#listing'

  def parse_item(self,item):
    title=""
    url=""
    title_element = item.find('h3', class_='c-card__titulo')
    if title_element:
      parent_a = title_element.find_parent('a')
      url = parent_a.get('href')
      title = title_element.get_text(strip=True)

    description = item.find('div', class_='c-card__descripcion')
    if description:
      decription = ' '.join(description.stripped_strings)

    category = item.find('div', class_='c-tax')
    if category:
      category = ' | '.join(category.stripped_strings)

    date = item.find('span', class_='c-card__fecha-txt')
    if date:
      date = date.get_text(strip=True)

    new_item = {
        "title": title,
        "description": description,
        "url": url,
        "category": category,
        "date": date
    }
    return new_item

  def get_articles(self):
    articles = []
    for page in range(1,848):
        url = self.get_url(page)
        response = requests.get(url)
        if response.status_code == 200:
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')
            items = soup.find_all('div', class_='c-list__item')
            for item in items:
              new_item = self.parse_item(item)
              articles.append(new_item)

        else:
            print(f'Error al obtener la página {page}:', response.status_code)
    return articles

In [ ]:
scraper = FactCheckScraper()
articles = scraper.get_articles()

In [ ]:
len(articles)

10161

In [ ]:
df = pd.DataFrame(articles)
df.head()

,title,description,url,category,date
0,"La inflación de abril de 2024 fue del 8,8%, se...",[[El Instituto Nacional de Estadística y Censo...,https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024"
1,Inflación 2024: qué son los precios relativos ...,[[[Es la relación entre el precio de un bien d...,https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024"
2,Una tormenta solar provocó auroras polares dur...,[[[Las auroras polares que se vieron en difere...,https://chequeado.com/ultimas-noticias/una-tor...,Viral | Falso,"14 mayo, 2024"
3,Vouchers educativos: cómo saber si obtuve el b...,[[[Más de un millón de estudiantes se encuentr...,https://chequeado.com/el-explicador/vouchers-e...,Educación | El Explicador,"13 mayo, 2024"
4,Debaten la privatización de Aerolíneas: tiene ...,[[[El proyecto de Ley Bases que tiene media sa...,https://chequeado.com/el-explicador/debaten-la...,Economía | El Explicador,"14 mayo, 2024"


In [ ]:
df.isnull().sum()

title             0
description    8575
url               0
category          0
date              0
dtype: int64

In [ ]:
df_subset= df.drop(columns=["description"])

In [ ]:
df_subset

,title,url,category,date
0,"La inflación de abril de 2024 fue del 8,8%, se...",https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024"
1,Inflación 2024: qué son los precios relativos ...,https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024"
2,Una tormenta solar provocó auroras polares dur...,https://chequeado.com/ultimas-noticias/una-tor...,Viral | Falso,"14 mayo, 2024"
3,Vouchers educativos: cómo saber si obtuve el b...,https://chequeado.com/el-explicador/vouchers-e...,Educación | El Explicador,"13 mayo, 2024"
4,Debaten la privatización de Aerolíneas: tiene ...,https://chequeado.com/el-explicador/debaten-la...,Economía | El Explicador,"14 mayo, 2024"
...,...,...,...,...
10156,Cristina Fernández: la matrícula escolar creci...,https://chequeado.com/el-explicador/el-gran-si...,Educación | Investigaciones,"23 junio, 2010"
10157,Reforma electoral: una ley con mala prensa,https://chequeado.com/el-explicador/ilos-chico...,El Explicador,"10 junio, 2010"
10158,La sociedad argentina no está orgullosa de sus...,https://chequeado.com/el-explicador/la-socieda...,El Explicador,"8 junio, 2010"
10159,"Mykonos on sale, ¿cuánto sale?",https://chequeado.com/el-explicador/mykonos-on...,Investigaciones,"2 junio, 2010"


In [ ]:
df_subset.to_csv('noticias.csv', index=False)

### Google Fact Checking

In [ ]:
class FactCheckGogle():

  def parse_item(self,claim):
    text = claim.get('text', 'N/A')
    claimant = claim.get('claimant', 'N/A')
    claim_date = claim.get('claimDate', 'N/A')
    review = claim.get('claimReview', [])[0] if claim.get('claimReview') else {}
    title = review.get('title', 'N/A')
    url = review.get('url', 'N/A')
    review_date = review.get('reviewDate', 'N/A')
    publisher = review.get('publisher', {}).get('name', 'N/A')

    new_item = {
        "claim": text,
        "claimant": claimant,
        "claim_date": claim_date,
        "review_title": title,
        "review_url": url,
        "review_date": review_date,
        "publisher": publisher
    }
    return new_item

  def get_articles(self,url,api_key):
    items = []
    params = {
          'query': 'Argentina',
          'key': api_key,
          'languageCode': 'es',
          'maxAgeDays': 10000,
          'pageSize':10000
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
      data = response.json()
      for claim in data.get('claims', []):
        item = self.parse_item(claim)
        items.append(item)

      return items
    else:
        print(f"Error: {response.status_code}")


In [ ]:
api_key = "API-KEY"
url = 'https://factchecktools.googleapis.com/v1alpha1/claims:search'

In [ ]:
fact_google = FactCheckGogle()
items = fact_google.get_articles(url,api_key)

In [ ]:
len(items)

615

In [ ]:
df_google = pd.DataFrame(items)
df_google

,claim,claimant,claim_date,review_title,review_url,review_date,publisher
0,Clarín: “Wos dejará de dar conciertos en Argen...,Anónimo,2024-05-05T00:00:00Z,"(Imagen) ""Wos dejará de dar conciertos en Arge...",https://www.fastcheck.cl/2024/05/08/imagen-wos...,2024-05-08T00:00:00Z,Fast Check 🔍
1,Biden dijo que el presidente argentino entregó...,N/A,N/A,Biden no dijo que el presidente argentino entr...,https://www.univision.com/noticias/biden-dijo-...,2024-04-24T00:00:00Z,Univision
2,Sergio Massa dejó una inflación anualizada del...,Varias fuentes,2024-05-07T00:00:00Z,Es falso que Sergio Massa haya dejado su carte...,https://factual.afp.com/doc.afp.com.34RJ226,2024-05-13T21:42:00Z,AFP Factual
3,En agosto de 2023 aumentó la cansasta básica e...,Varias fuentes,2024-04-04T00:00:00Z,Datos sobre la canasta básica en Argentina cir...,https://factual.afp.com/doc.afp.com.34R767P,2024-05-09T21:18:00Z,AFP Factual
4,"Clarín: Wos dejará de dar conciertos ""hasta qu...",Varias fuentes,2024-05-03T00:00:00Z,Clarín no publicó que Wos dejará de tocar en A...,https://factual.afp.com/doc.afp.com.34R38NN,2024-05-07T22:23:00Z,AFP Factual
...,...,...,...,...,...,...,...
610,"Médicos por la Verdad, Mariano Arriaga, Luis M...",Varias fuentes,2021-03-12T00:00:00Z,Médicos argentinos cuestionan muertes por covi...,https://factual.afp.com/medicos-argentinos-cue...,2021-03-16T22:01:00Z,AFP Factual
611,"El vídeo llamado ""Argentina lo está logrando"" ...",N/A,N/A,"No, un archivo llamado ""Argentina lo está logr...",https://maldita.es/malditobulo/20201007/argent...,2021-08-25T00:00:00Z,Maldita.es
612,Debate en el Congreso argentino sobre si las m...,FactualMultiple sources,2017-07-09T00:00:00Z,Este video no fue grabado en 1947 durante el d...,https://factual.afp.com/este-video-no-fue-grab...,2018-09-28T19:09:00Z,AFP Factual
613,El proyecto para renovar el Código Penal de Ar...,Varias fuentes,2020-08-20T00:00:00Z,El proyecto para renovar el Código Penal de Ar...,https://factual.afp.com/el-proyecto-para-renov...,2020-08-28T00:22:00Z,AFP Factual


In [ ]:
df_google.isnull().sum()

claim           0
claimant        0
claim_date      0
review_title    0
review_url      0
review_date     0
publisher       0
dtype: int64

In [ ]:
df_google.to_csv('google_noticias.csv', index=False)

### Join Datasets

In [69]:
news = pd.read_csv("./noticias.csv")
news.head()

,title,url,category,date
0,"La inflación de abril de 2024 fue del 8,8%, se...",https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024"
1,Inflación 2024: qué son los precios relativos ...,https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024"
2,Una tormenta solar provocó auroras polares dur...,https://chequeado.com/ultimas-noticias/una-tor...,Viral | Falso,"14 mayo, 2024"
3,Vouchers educativos: cómo saber si obtuve el b...,https://chequeado.com/el-explicador/vouchers-e...,Educación | El Explicador,"13 mayo, 2024"
4,Debaten la privatización de Aerolíneas: tiene ...,https://chequeado.com/el-explicador/debaten-la...,Economía | El Explicador,"14 mayo, 2024"


In [71]:
news["source"] = "Chequeado"

In [72]:
news.head()

,title,url,category,date,source
0,"La inflación de abril de 2024 fue del 8,8%, se...",https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024",Chequeado
1,Inflación 2024: qué son los precios relativos ...,https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024",Chequeado
2,Una tormenta solar provocó auroras polares dur...,https://chequeado.com/ultimas-noticias/una-tor...,Viral | Falso,"14 mayo, 2024",Chequeado
3,Vouchers educativos: cómo saber si obtuve el b...,https://chequeado.com/el-explicador/vouchers-e...,Educación | El Explicador,"13 mayo, 2024",Chequeado
4,Debaten la privatización de Aerolíneas: tiene ...,https://chequeado.com/el-explicador/debaten-la...,Economía | El Explicador,"14 mayo, 2024",Chequeado


In [73]:
def convert_date(fecha):
    meses = {
        'enero': '01',
        'febrero': '02',
        'marzo': '03',
        'abril': '04',
        'mayo': '05',
        'junio': '06',
        'julio': '07',
        'agosto': '08',
        'septiembre': '09',
        'octubre': '10',
        'noviembre': '11',
        'diciembre': '12'
    }
    dia, mes, anio = fecha.split()
    mes = mes.strip(',')

    mes_num = meses[mes]

    nueva_fecha = f"{dia.zfill(2)}/{mes_num}/{anio}"

    return nueva_fecha


In [74]:
news['fecha_convertida'] = news['date'].apply(convert_date)
news['fecha_convertida'] = pd.to_datetime(news['fecha_convertida'],dayfirst=True).dt.strftime('%d/%m/%Y')


In [75]:
news.head()

,title,url,category,date,source,fecha_convertida
0,"La inflación de abril de 2024 fue del 8,8%, se...",https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024",Chequeado,14/05/2024
1,Inflación 2024: qué son los precios relativos ...,https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024",Chequeado,14/05/2024
2,Una tormenta solar provocó auroras polares dur...,https://chequeado.com/ultimas-noticias/una-tor...,Viral | Falso,"14 mayo, 2024",Chequeado,14/05/2024
3,Vouchers educativos: cómo saber si obtuve el b...,https://chequeado.com/el-explicador/vouchers-e...,Educación | El Explicador,"13 mayo, 2024",Chequeado,13/05/2024
4,Debaten la privatización de Aerolíneas: tiene ...,https://chequeado.com/el-explicador/debaten-la...,Economía | El Explicador,"14 mayo, 2024",Chequeado,14/05/2024


In [76]:
news.value_counts("fecha_convertida")

fecha_convertida
17/02/2015    374
17/08/2021    254
12/08/2021    190
18/08/2021    176
13/08/2021    138
             ... 
10/02/2016      1
10/02/2018      1
21/05/2019      1
21/05/2018      1
01/01/2013      1
Name: count, Length: 3042, dtype: int64

In [86]:
google_news = pd.read_csv("./google_noticias.csv")
google_news.head()

,claim,claimant,claim_date,review_title,review_url,review_date,publisher
0,Clarín: “Wos dejará de dar conciertos en Argen...,Anónimo,2024-05-05T00:00:00Z,"(Imagen) ""Wos dejará de dar conciertos en Arge...",https://www.fastcheck.cl/2024/05/08/imagen-wos...,2024-05-08T00:00:00Z,Fast Check 🔍
1,Biden dijo que el presidente argentino entregó...,NaN,NaN,Biden no dijo que el presidente argentino entr...,https://www.univision.com/noticias/biden-dijo-...,2024-04-24T00:00:00Z,Univision
2,Sergio Massa dejó una inflación anualizada del...,Varias fuentes,2024-05-07T00:00:00Z,Es falso que Sergio Massa haya dejado su carte...,https://factual.afp.com/doc.afp.com.34RJ226,2024-05-13T21:42:00Z,AFP Factual
3,En agosto de 2023 aumentó la cansasta básica e...,Varias fuentes,2024-04-04T00:00:00Z,Datos sobre la canasta básica en Argentina cir...,https://factual.afp.com/doc.afp.com.34R767P,2024-05-09T21:18:00Z,AFP Factual
4,"Clarín: Wos dejará de dar conciertos ""hasta qu...",Varias fuentes,2024-05-03T00:00:00Z,Clarín no publicó que Wos dejará de tocar en A...,https://factual.afp.com/doc.afp.com.34R38NN,2024-05-07T22:23:00Z,AFP Factual


In [88]:
google_news.rename(columns={'review_title': 'title','review_url': 'url','review_date': 'date','publisher': 'source'}, inplace=True)
google_news["category"] = "Google Fact Check"

In [89]:
google_news_subset = google_news.loc[:,["title","url","category","date","source"]]
google_news_subset.head()

,title,url,category,date,source
0,"(Imagen) ""Wos dejará de dar conciertos en Arge...",https://www.fastcheck.cl/2024/05/08/imagen-wos...,Google Fact Check,2024-05-08T00:00:00Z,Fast Check 🔍
1,Biden no dijo que el presidente argentino entr...,https://www.univision.com/noticias/biden-dijo-...,Google Fact Check,2024-04-24T00:00:00Z,Univision
2,Es falso que Sergio Massa haya dejado su carte...,https://factual.afp.com/doc.afp.com.34RJ226,Google Fact Check,2024-05-13T21:42:00Z,AFP Factual
3,Datos sobre la canasta básica en Argentina cir...,https://factual.afp.com/doc.afp.com.34R767P,Google Fact Check,2024-05-09T21:18:00Z,AFP Factual
4,Clarín no publicó que Wos dejará de tocar en A...,https://factual.afp.com/doc.afp.com.34R38NN,Google Fact Check,2024-05-07T22:23:00Z,AFP Factual


In [90]:
google_news_subset['fecha_convertida'] = pd.to_datetime(google_news_subset['date']).dt.strftime('%d/%m/%Y')
google_news_subset.head()

,title,url,category,date,source,fecha_convertida
0,"(Imagen) ""Wos dejará de dar conciertos en Arge...",https://www.fastcheck.cl/2024/05/08/imagen-wos...,Google Fact Check,2024-05-08T00:00:00Z,Fast Check 🔍,08/05/2024
1,Biden no dijo que el presidente argentino entr...,https://www.univision.com/noticias/biden-dijo-...,Google Fact Check,2024-04-24T00:00:00Z,Univision,24/04/2024
2,Es falso que Sergio Massa haya dejado su carte...,https://factual.afp.com/doc.afp.com.34RJ226,Google Fact Check,2024-05-13T21:42:00Z,AFP Factual,13/05/2024
3,Datos sobre la canasta básica en Argentina cir...,https://factual.afp.com/doc.afp.com.34R767P,Google Fact Check,2024-05-09T21:18:00Z,AFP Factual,09/05/2024
4,Clarín no publicó que Wos dejará de tocar en A...,https://factual.afp.com/doc.afp.com.34R38NN,Google Fact Check,2024-05-07T22:23:00Z,AFP Factual,07/05/2024


In [91]:
all_news = pd.concat([news, google_news_subset],ignore_index=True)
all_news

,title,url,category,date,source,fecha_convertida
0,"La inflación de abril de 2024 fue del 8,8%, se...",https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024",Chequeado,14/05/2024
1,Inflación 2024: qué son los precios relativos ...,https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,"14 mayo, 2024",Chequeado,14/05/2024
2,Una tormenta solar provocó auroras polares dur...,https://chequeado.com/ultimas-noticias/una-tor...,Viral | Falso,"14 mayo, 2024",Chequeado,14/05/2024
3,Vouchers educativos: cómo saber si obtuve el b...,https://chequeado.com/el-explicador/vouchers-e...,Educación | El Explicador,"13 mayo, 2024",Chequeado,13/05/2024
4,Debaten la privatización de Aerolíneas: tiene ...,https://chequeado.com/el-explicador/debaten-la...,Economía | El Explicador,"14 mayo, 2024",Chequeado,14/05/2024
...,...,...,...,...,...,...
10771,Médicos argentinos cuestionan muertes por covi...,https://factual.afp.com/medicos-argentinos-cue...,Google Fact Check,2021-03-16T22:01:00Z,AFP Factual,16/03/2021
10772,"No, un archivo llamado ""Argentina lo está logr...",https://maldita.es/malditobulo/20201007/argent...,Google Fact Check,2021-08-25T00:00:00Z,Maldita.es,25/08/2021
10773,Este video no fue grabado en 1947 durante el d...,https://factual.afp.com/este-video-no-fue-grab...,Google Fact Check,2018-09-28T19:09:00Z,AFP Factual,28/09/2018
10774,El proyecto para renovar el Código Penal de Ar...,https://factual.afp.com/el-proyecto-para-renov...,Google Fact Check,2020-08-28T00:22:00Z,AFP Factual,28/08/2020


In [92]:
all_news_subset = all_news.loc[:,["title","url","category","fecha_convertida","source"]]
all_news_subset.rename(columns={'fecha_convertida': 'date'}, inplace=True)
all_news_subset.head()

,title,url,category,date,source
0,"La inflación de abril de 2024 fue del 8,8%, se...",https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,14/05/2024,Chequeado
1,Inflación 2024: qué son los precios relativos ...,https://chequeado.com/el-explicador/inflacion-...,Economía | El Explicador,14/05/2024,Chequeado
2,Una tormenta solar provocó auroras polares dur...,https://chequeado.com/ultimas-noticias/una-tor...,Viral | Falso,14/05/2024,Chequeado
3,Vouchers educativos: cómo saber si obtuve el b...,https://chequeado.com/el-explicador/vouchers-e...,Educación | El Explicador,13/05/2024,Chequeado
4,Debaten la privatización de Aerolíneas: tiene ...,https://chequeado.com/el-explicador/debaten-la...,Economía | El Explicador,14/05/2024,Chequeado


In [93]:
all_news_subset.to_csv('all_news.csv', index=False)